In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# File with helper methods
import tools

%matplotlib inline

In [65]:
df_trips_stops = pd.read_csv('data/2_processed/df_trips_stops_ready.csv')

In [66]:
 df_trips_stops.drop_duplicates(['rt_trip_id', 'rt_stop_id', 'departure'], inplace=True)

In [67]:
print(df_trips_stops.shape)
df_trips_stops.head()

(587632, 8)


,rt_trip_id,rt_stop_id,stop_offset,departure,lat,lng,distance_prev,departure_date
0,201955859,51843,30,1568606460,45.541968,-73.590304,162.0,2019-09-16 04:01:00
1,201955859,61653,37,1568606820,45.531186,-73.597507,289.0,2019-09-16 04:07:00
2,201955861,51755,3,1568607347,45.536114,-73.593775,181.0,2019-09-16 04:15:47
3,201955861,61579,4,1568607385,45.537211,-73.592786,144.0,2019-09-16 04:16:25
4,201955862,53151,0,1568609160,45.582726,-73.542559,0.0,2019-09-16 04:46:00


Frame before dropping duplicates : (600323, 7)

In [68]:
df_trips_stops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 587632 entries, 0 to 589447
Data columns (total 8 columns):
rt_trip_id        587632 non-null int64
rt_stop_id        587632 non-null int64
stop_offset       587632 non-null int64
departure         587632 non-null int64
lat               587632 non-null float64
lng               587632 non-null float64
distance_prev     587632 non-null float64
departure_date    587632 non-null object
dtypes: float64(3), int64(4), object(1)
memory usage: 40.3+ MB


### By stop

In [41]:
 df_grouped_by_stop = df_trips_stops.groupby('rt_stop_id')
len(df_grouped_by_stop.groups.keys())

8756

In [42]:
type(df_grouped_by_stop.size())
df_grouped_by_stop.size().sort_values()
df_grouped_by_stop.size().describe()

count    8756.000000
mean       67.322407
std        46.510097
min         1.000000
25%        37.000000
50%        49.000000
75%        94.000000
max       360.000000
dtype: float64

In [44]:
# df_grouped_by_stop.size()

In [47]:
# Calculate how many stops belong only to one route
list_routes = list(df_grouped_by_stop.size().values)
list_route_indexes = list(df_grouped_by_stop.size().index)

print(min(df_grouped_by_stop.size().values))

list_routes.count(min(df_grouped_by_stop.size().values))

1


34

A maximum of 365 different routes pass by a single stop. Theres 18 stops that form part of only one route. In average 68.56 routes per stop. We need to see if these values are true or if we have more than different instances of the same trip under specific stops. Let's study the stop that has 365 rows under it's group.

In [48]:
list_route_indexes[list_routes.index(360)]

53287

The stop which has 365 different records is : 53287. Let's extract its group to take a closer look.

In [64]:
df_per_stop = df_grouped_by_stop.get_group(30)
print(df_per_stop.shape)
df_per_stop.head()

(1, 7)


,rt_trip_id,rt_stop_id,stop_offset,departure,lat,lng,distance_prev
225987,19S-GLOBAUX-01-S_1_2,30,13,1568627340,45.511032,-73.564898,424.0


In [50]:
df_per_stop['rt_trip_id'].nunique()

360

we have only 360 unique stop ids. Let's extract look at the duplicated ones to understand its nature.

In [51]:
# x = df_per_stop['rt_trip_id']
y = list(set(df_per_stop['rt_trip_id']))

duplicates = seen = set()
uniq = [x for x in df_per_stop['rt_trip_id'] if x not in seen and not seen.add(x)] 

In [52]:
def find_duplicates(duped_list):
    seen = {}
    dupes = []

    for x in duped_list:
        if x not in seen:
            seen[x] = 1
        else:
            if seen[x] == 1:
                dupes.append(x)
            seen[x] += 1
    return dupes

In [53]:
duplicates = find_duplicates(df_per_stop['rt_trip_id'].values)

In [54]:
duplicates

[]

In [55]:
for dup_id in duplicates:
    print('Checking trip id: ' + str(dup_id))
    print('NUmber of duplicated ids: ' + str(list(df_per_stop['rt_trip_id'].values).count(dup_id)))
    duped_ids = df_per_stop.loc[df_per_stop['rt_trip_id'] == dup_id]
    print(duped_ids)

In [56]:
duplicateRowsDF = df_per_stop[df_per_stop.duplicated(['rt_trip_id'], keep='first')]

In [57]:
print(duplicateRowsDF.shape)
duplicateRowsDF.head()
# ['203580218', '203579651', '203582040', '203579798', '203582102']

(0, 7)


,rt_trip_id,rt_stop_id,stop_offset,departure,lat,lng,distance_prev


Looks like we have 5 different trips for this stop that have duplicated columns. This means that we could have ingested the data twice. We should eliminate all these duplicates before proceding.

Let's see if we can do so directly.

In [58]:
df_trip_stop_dupes = df_trips_stops[df_trips_stops.duplicated(['rt_trip_id', 'rt_stop_id', 'departure'])] 

In [59]:
print(df_trip_stop_dupes.shape)
df_trip_stop_dupes.head()

(0, 7)


,rt_trip_id,rt_stop_id,stop_offset,departure,lat,lng,distance_prev


## By both trip and stop

In [60]:
 df_grouped_by_stop_and_trip = df_trips_stops.groupby(['rt_stop_id', 'rt_trip_id'])

In [61]:
# df_grouped_by_stop_and_trip.groups.keys()
df_grouped_by_stop_and_trip.size()

rt_stop_id  rt_trip_id                    
3           19S-GLOBAUX-01-S_2_0_205238755    1
8           19S-GLOBAUX-01-S_2_0              1
9           19S-GLOBAUX-01-S_2_0              1
10          19S-GLOBAUX-01-S_2_0              1
14          19S-GLOBAUX-01-S_2_0              1
            19S-GLOBAUX-01-S_2_0_205238755    1
            19S-GLOBAUX-01-S_2_1              1
17          19S-GLOBAUX-01-S_2_1              1
26          19S-GLOBAUX-01-S_1_2              1
30          19S-GLOBAUX-01-S_1_2              1
31          19S-GLOBAUX-01-S_1_2              1
32          19S-GLOBAUX-01-S_1_2              1
33          19S-GLOBAUX-01-S_1_2              1
36          19S-GLOBAUX-01-S_1_2              1
            19S-GLOBAUX-01-S_2_0              1
            19S-GLOBAUX-01-S_2_0_205238755    1
39          19S-GLOBAUX-01-S_1_2              1
41          19S-GLOBAUX-01-S_1_2              1
46          19S-GLOBAUX-01-S_2_0_205238755    1
53          19S-GLOBAUX-01-S_2_0             

In [28]:
df_grouped_by_stop.get_group(41).shape

(14, 7)

In [29]:
df_grouped_by_stop.get_group(41)

,rt_trip_id,rt_stop_id,stop_offset,departure,lat,lng,distance_prev
225981,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
281114,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
308037,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
308560,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
318122,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
382517,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
401267,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
436109,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
436118,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0
509502,19S-GLOBAUX-01-S_1_2,41,2,1568626380,45.45701,-73.581688,1203.0


we have so many duplicated rows. If two different fows have exactly the same trip and stop id and also departure date that means that those are dupliated records. Let's delete them and check how things behave.

In [36]:
 df_grouped_by_stop.drop_duplicates(['rt_trip_id', 'rt_stop_id', 'departure'], inplace=True)

AttributeError: Cannot access callable attribute 'drop_duplicates' of 'DataFrameGroupBy' objects, try using the 'apply' method